# Variables

# Config module search path

In [ ]:
import sys
from pathlib import Path

# Get my_package directory path from Notebook
parent_dir = str(Path().resolve().parents[1])

# Add to sys.path
if parent_dir not in sys.path:
  sys.path.insert(0, parent_dir)

from pprint import pprint
pprint(sys.path)

In [ ]:
import re
from pprint import pprint

from textsummary.prompts import Prompter
from textsummary.summary import proxy_summary

# Prompters

## Simple Prompter

In [ ]:
class SimplePrompter(Prompter):
  def __init__(self, min=200, max=250, num_sentence=6, sentence_length=40):
    super().__init__()
    self._template = '''{text}
TL;DR 请为以上字提供摘要，要求如下:
1. 摘要以中文作答;
2. 摘要不少于 {min} 字，不超过 {max} 字;
3. 摘要包含 {num_sentence} 句话, 每句话 {sentence_length} 字左右
4. 摘要中每句话以中文句号作为每句话的结束;
5. 摘要中不要包含无关文字, 最后一句摘要不使用总结的口吻;
'''.format(
      text='{text}',
      min=min,
      max=max,
      num_sentence=num_sentence,
      sentence_length=sentence_length
    )

  def prompt(self, text: str) -> str:
    return self._template.format(text=text)

  def summaries(self, response: str) -> list[str]:
    return self.instructions(response)

  def instructions(self, response: str) -> list[str]:
    return [s for s in re.split(r'。|；', response) if len(s) > 0]

## Scene Prompter

In [ ]:
class ScenePrompter(Prompter):
  def __init__(self, num_scene=6, num_keyword=1, sentence_length=40):
    super().__init__()
    self._template = '''{text}
我想根据这段文字做一个营销短视频，作为营销大师，请帮我设计出{num_scene}个适合短视频的场景。
每个场景提炼{num_keyword}个关键词，再写{sentence_length}个字左右的抖音风格文案。
其中第一个场景是整体介绍，不要出现原文中没有的地名和产品名。
请使用中文输出，输出时的格式为: "场景:, 关键词:, 文案:"。
输出时请直接给出结果，不要包含无关语句。'''.format(
      text='{text}',
      num_scene=num_scene,
      num_keyword=num_keyword,
      sentence_length=sentence_length
    )

  def prompt(self, text: str) -> str:
    return self._template.format(text=text)

  def summaries(self, response: str) -> list[str]:
    return [re.split(r':|：', s)[-1].strip() for s in self.instructions(response)]

  def instructions(self, response: str) -> list[str]:
    return ['场景'+s.strip() for s in response.split('场景') if len(s.strip()) > 0]

In [ ]:
text = '''
  软装设计，是设计一种生活方式，创造一种更为舒适的生活状态。
  无需刻意新奇，无关美丑新旧，只为营造家的舒适感，对空间的归属感，以及身心愉悦的装饰氛围。
  本期方案，利用干净的配色、明晰的线条、丰富的造型，以及充足的采光，让空间充满元气，生活充满幸福感。
  客餐厅一体化设计，空间开阔感极大提升。简约的无主灯设计，灯带与筒灯实现均匀照明与局部重点照明，更具氛围感。
  客餐厅两侧窗户大面积自然采光，则让空间时刻充满自然气息。
  客餐厅配色以无彩色为主，浅灰色搭配棕灰色的大面积墙体，干净简洁。肤感膜、细格栅与波浪板的应用，则让空间材质与造型更为丰富。
  黑色金属线条的加入，能灵活地设计墙咔布局与造型，金属质感，时尚精致，令装饰的灵动感与趣味性更上一层。
  窗帘采用拼色款，与木纹墙咔的棕灰色相呼应，色彩和谐的同时，良好的遮光性与垂顺度，进一步提升空间舒适度与美感。
  主卧延续整体的设计风格，大面积无彩色干净简洁的基调下，利用木纹墙咔的棕色系，加以点缀。
  床体选用质感十足的绒面材质，柔软温和，棕色系的加持，更让舒适指数大为提升。
  壁上观背景以及床头挂画，作为点睛之笔，赋予空间时尚艺术性。
  整体给人的感觉，在现代简约之下，有一点点复古怀旧，一点点清新自然。
  儿童房的打造，则是在原有风格的基础上，点缀了充满生机的绿色。床头挂画与懒人沙发，深绿色装饰清新自然。
  衣柜则是淡淡的湖水绿，让空间的氛围更加轻松，整体是现代与森系结合的舒适空间。
  为 YAKINO 雅琪诺点赞。
'''

summaries, instructions = proxy_summary(text, ScenePrompter())
print('Summaries:')
for i, str in enumerate(summaries):
  print(f'[{i}] {str}')
print('\n')
print('Instructions:')
for i, str in enumerate(instructions):
  print(f'[{i}] {str}')
